In [45]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection details
DB_HOST = "localhost"
DB_PORT = 5433  # Port you mapped in docker-compose.yaml
DB_NAME = "ny_taxi"
DB_USER = "postgres"
DB_PASS = "postgres"

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection to PostgreSQL successful!")
except Exception as e:
    print(f"Error connecting to PostgreSQL: {e}")


Connection to PostgreSQL successful!


In [46]:
# show first 5 rows of the green_taxi_trips table
query = """
SELECT *
FROM green_taxi_trips
"""
green_taxi_trips = pd.read_sql(query, engine)
green_taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               387007 non-null  float64
 1   lpep_pickup_datetime   476386 non-null  object 
 2   lpep_dropoff_datetime  476386 non-null  object 
 3   store_and_fwd_flag     387007 non-null  object 
 4   RatecodeID             387007 non-null  float64
 5   PULocationID           476386 non-null  int64  
 6   DOLocationID           476386 non-null  int64  
 7   passenger_count        387007 non-null  float64
 8   trip_distance          476386 non-null  float64
 9   fare_amount            476386 non-null  float64
 10  extra                  476386 non-null  float64
 11  mta_tax                476386 non-null  float64
 12  tip_amount             476386 non-null  float64
 13  tolls_amount           476386 non-null  float64
 14  ehail_fee              0 non-null   

In [47]:
green_taxi_trips.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1.0,112,196,1.0,5.88,18.0,0.50,0.5,0.00,0.0,None,0.3,19.30,2.0,1.0,0.0
1,1.0,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1.0,43,263,1.0,0.80,5.0,3.25,0.5,0.00,0.0,None,0.3,9.05,2.0,1.0,0.0
2,1.0,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1.0,255,228,2.0,7.50,21.5,0.50,0.5,0.00,0.0,None,0.3,22.80,2.0,1.0,0.0
3,1.0,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1.0,181,181,1.0,0.90,5.5,0.50,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.0
4,2.0,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1.0,97,188,1.0,2.52,10.0,0.50,0.5,2.26,0.0,None,0.3,13.56,1.0,1.0,0.0


In [48]:
# read taxi_zones table into a DataFrame
query = """
SELECT *
FROM taxi_zones
"""
taxi_zones = pd.read_sql(query, engine)
taxi_zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [49]:
taxi_zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


## Question 1. Understanding docker first run
Run docker with the python:3.12.8 image in an interactive mode, use the entrypoint bash.

What's the version of pip in the image?

24.3.1


## Question 2. Understanding Docker networking and docker-compose
Given the following docker-compose.yaml, what is the hostname and port that pgadmin should use to connect to the postgres database?
```yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

db:5432


## Question 3. Trip Segmentation Count
During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

1. Up to 1 mile
2. In between 1 (exclusive) and 3 miles (inclusive),
3. In between 3 (exclusive) and 7 miles (inclusive),
4. In between 7 (exclusive) and 10 miles (inclusive),
5. Over 10 miles


In [61]:
import pandas as pd

# Define bins and labels
distance_bins = [0, 1, 3, 7, 10, float("inf")]
distance_labels = ["Up to 1 mile", "1-3 miles", "3-7 miles", "7-10 miles", "Over 10 miles"]

# Filter trips within the date range
green_taxi_trips["lpep_pickup_datetime"] = pd.to_datetime(green_taxi_trips["lpep_pickup_datetime"])
filtered_trips = green_taxi_trips[
    (green_taxi_trips["lpep_pickup_datetime"] >= "2019-10-01 00:00:00") &
    (green_taxi_trips["lpep_pickup_datetime"] < "2019-11-01 00:00:00")
    ].copy()

# Categorize trips by distance
filtered_trips["distance_category"] = pd.cut(
    filtered_trips["trip_distance"],
    bins=distance_bins,
    labels=distance_labels,
    include_lowest=True  # Include 0 in the first bin
)

# Count trips in each category
trip_counts = filtered_trips["distance_category"].value_counts(sort=False)

# Display results in the same order as the question
for label, count in zip(distance_labels, trip_counts):
    print(f"{label}: {count}")


Up to 1 mile: 104828
1-3 miles: 198995
3-7 miles: 109642
7-10 miles: 27686
Over 10 miles: 35201


In [62]:
# Define conditions for each bin
conditions = [
    (filtered_trips["trip_distance"] >= 0) & (filtered_trips["trip_distance"] <= 1),  # Up to 1 mile
    (filtered_trips["trip_distance"] > 1) & (filtered_trips["trip_distance"] <= 3),  # 1-3 miles
    (filtered_trips["trip_distance"] > 3) & (filtered_trips["trip_distance"] <= 7),  # 3-7 miles
    (filtered_trips["trip_distance"] > 7) & (filtered_trips["trip_distance"] <= 10), # 7-10 miles
    (filtered_trips["trip_distance"] > 10)                                          # Over 10 miles
]

# Define labels for each bin
labels = ["Up to 1 mile", "1-3 miles", "3-7 miles", "7-10 miles", "Over 10 miles"]

# Apply the conditions to create a new column
filtered_trips["distance_category_manual"] = None  # Initialize with None
for condition, label in zip(conditions, labels):
    filtered_trips.loc[condition, "distance_category_manual"] = label

# Verify the counts in each bin
manual_counts = filtered_trips["distance_category_manual"].value_counts()
print(manual_counts)


distance_category_manual
1-3 miles        198995
3-7 miles        109642
Up to 1 mile     104828
Over 10 miles     35201
7-10 miles        27686
Name: count, dtype: int64


In [65]:
# find longest trip for each day, Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

# Extract date from pickup datetime
filtered_trips["pickup_date"] = filtered_trips["lpep_pickup_datetime"].dt.date

# Find the longest trip for each day
longest_trips = filtered_trips.groupby("pickup_date")["trip_distance"].max()

# Find the date with the longest trip
longest_trip_date = longest_trips.idxmax()
longest_trip_distance = longest_trips.max()

print(f"Date with the longest trip: {longest_trip_date}")
print(f"Longest trip distance: {longest_trip_distance} miles")


Date with the longest trip: 2019-10-31
Longest trip distance: 515.89 miles


In [68]:
# Question 5. Three biggest pickup zones Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?
# Consider only lpep_pickup_datetime when filtering by date.

# Filter trips for the specific date
date = "2019-10-18"
filtered_trips_date = green_taxi_trips[
    (green_taxi_trips["lpep_pickup_datetime"].dt.date == pd.to_datetime(date).date())
]

# Find the top pickup locations by total amount
top_pickup_zones = filtered_trips_date.groupby("PULocationID")["total_amount"].sum().nlargest(3)

print("Top pickup locations with over $13,000 in total_amount:")
for location_id, total_amount in top_pickup_zones.items():
    location_name = taxi_zones.loc[taxi_zones["LocationID"] == location_id, "Zone"].values[0]
    print(f"Location ID: {location_id}, Name: {location_name}, Total Amount: ${total_amount:.2f}")







Top pickup locations with over $13,000 in total_amount:
Location ID: 74, Name: East Harlem North, Total Amount: $18686.68
Location ID: 75, Name: East Harlem South, Total Amount: $16797.26
Location ID: 166, Name: Morningside Heights, Total Amount: $13029.79


In [69]:
# Question 6. Largest tip
# For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?
#
# Note: it's tip , not trip
#
# We need the name of the zone, not the ID.

# Filter trips for the specific zone and month
zone_name = "East Harlem North"
filtered_trips_zone = green_taxi_trips[
    (green_taxi_trips["PULocationID"] == taxi_zones.loc[taxi_zones["Zone"] == zone_name, "LocationID"].values[0]) &
    (green_taxi_trips["lpep_pickup_datetime"].dt.month == 10)
]

# Find the drop off zone with the largest tip
largest_tip_dropoff_zone = filtered_trips_zone.loc[filtered_trips_zone["tip_amount"].idxmax(), "DOLocationID"]
dropoff_zone_name = taxi_zones.loc[taxi_zones["LocationID"] == largest_tip_dropoff_zone, "Zone"].values[0]

print(f"Drop off zone with the largest tip for passengers picked up in 'East Harlem North': {dropoff_zone_name}")




Drop off zone with the largest tip for passengers picked up in 'East Harlem North': JFK Airport
